In [1]:
import pandas as pd
import numpy as np


In [3]:
URLS = [
    'X:/Users/Dimam/Downloads/prices-eurosvet-rur-utf-8.csv',
    'X:/Users/Dimam/Downloads/prices-elektrostandard-rur-utf-8.csv',
    'X:/Users/Dimam/Downloads/prices-werkel-rur-utf-8.csv',
    'X:/Users/Dimam/Downloads/prices-minimir-rur-utf-8.csv'
]
dataframes = []
for url in URLS:
    dataframes.append(pd.read_csv(url, sep=';', encoding='utf-8'))


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,24,29,30,32,34,35,39,52,54,55,56,57,60,61,63,64,65,66,67,71,72,73,74,75,78,79,83,87,89,90,93,94,95,100,101,102,103,104,110,111,112,116,120,138,139,142,143,145,146,147,148,152,162,166,167,168,170,178,182,185,186,189,190,197,198,199,202,203,204,205,208,209,211,212,219,220,242,243,244,245,246,247,248,249) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
partner_names=["eurosvet","elektrostandard","werkel","minimir"]
d = {'id': [1, 2,3,4], 'code': partner_names,'name':partner_names}
partner_partner = pd.DataFrame(data=d)



In [12]:
partner_partner

,id,code,name
0,1,eurosvet,eurosvet
1,2,elektrostandard,elektrostandard
2,3,werkel,werkel
3,4,minimir,minimir


In [4]:
# make shop dataset
catalogue_product_headers = ['id', 'price',
                             'description', 'title', 'name', 'upc']
catalogue_product = pd.DataFrame([], columns=catalogue_product_headers)


In [5]:
def merge_columns(dataframes, col_name,):
    merged_column = []
    for dataframe in dataframes:
        merged_column.append(dataframe[col_name].to_numpy())
    return np.concatenate(merged_column)


In [6]:
what_copy = {
    'price': 'Цена',
    'description': 'Описание',
    'title': 'SEO название для сайта',
    'name': 'Наименование',
    'upc': 'Штрихкод'
    # 'slug':'SEO название',
}
for key in what_copy:
    catalogue_product[key] = merge_columns(dataframes, what_copy[key])
catalogue_product['id'] = np.arange(1, len(catalogue_product['id'])+1)
# shop_data['name']=merge_columns(dataframes,'Наименование')


In [7]:
catalogue_product

,id,price,description,title,name,upc
0,1,15500.0,NaN,Напольный светильник 5069 Treviso,5069 Treviso,5.901781e+12
1,2,26800.0,NaN,Хрустальная подвесная люстра 338/6 Strotskis S...,338/6 Strotskis Smart (318/6),4.690389e+12
2,3,40700.0,NaN,Потолочная люстра 351 Smart,351 Smart (570),4.690389e+12
3,4,24500.0,NaN,Потолочная люстра 350 Smart,350 Smart (569),4.690389e+12
4,5,45500.0,NaN,Потолочная люстра 349 Smart,349 Smart (553),4.690389e+12
...,...,...,...,...,...,...
9030,9031,1900.0,Бра со стеклянным плафоном 3457/1 хром/белый с...,Бра со стеклянным плафоном 3457/1 хром/белый,3457/1 хром/белый,4.607138e+12
9031,9032,8970.0,Люстра потолочная 3353/6 хром/белый предназнач...,Люстра потолочная 3353/6 хром/белый,3353/6 хром/белый,4.607138e+12
9032,9033,5760.0,Потолочная люстра 3353/3 хром/белый предназнач...,Потолочная люстра 3353/3 хром/белый,3353/3 хром/белый,4.607138e+12
9033,9034,6650.0,Торшер с тканевым абажуром 009/1T RDM в янтарн...,Торшер классический янтарный 009/1T RDM (янтарь),009/1T RDM (янтарь),4.607176e+12


In [7]:
# shop_data.to_csv('saved_shop.csv',index=False)

In [8]:
catalogue_productimage_headers = [
    'id', 'original', 'display_order', 'product_id']
catalogue_productimage = pd.DataFrame(
    [], columns=catalogue_productimage_headers)


In [8]:
def isNaN(num):
    return num != num


def make_list_of_images(row):
    merged_urls = []
    for i in range(len(row)):
        image_head = 'Изображение '+str(i)
        if image_head in row.keys() and not isNaN(row[image_head]):
            merged_urls.append(row[image_head])
    return merged_urls


def make_image_rows(to_dataframe: pd.DataFrame, from_dataframe):
    for i in range(len(from_dataframe.index)):
        image_urls = make_list_of_images(from_dataframe.iloc[i])
        for t in range(len(image_urls)):
            id = len(to_dataframe.index)+1
            to_dataframe.loc[id] = [id, image_urls[t], t, i]
    return to_dataframe


def fill_images_table(to_dataframe: pd.DataFrame, from_dataframes):
    for from_dataframe in from_dataframes:
        to_dataframe.merge(make_image_rows(to_dataframe, from_dataframe))
    return to_dataframe


# catalogue_productimage = fill_images_table(catalogue_productimage, dataframes)


In [ ]:
# catalogue_productimage.to_excel('saved_shop.xlsx',index=False)


In [ ]:
# stockrecord
# products prices

catalogue_product_headers = ['id', 'partner_sku',
                             'price_currency', 'price_excl_tax','price_reatail','cost_price','num_in_stock', 'partner_id', 'product_id']
catalogue_product = pd.DataFrame([], columns=catalogue_product_headers)